<a href="https://colab.research.google.com/github/zubinkm/Fedbatch/blob/main/trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from ._batch_bioreactor import BatchBioreactor
# failed to import batch bioreactor
from scipy.integrate import odeint
from thermosteam.reaction import Reaction, ParallelReaction
import matplotlib

In [ ]:
__all__ = ('Fermentation',)

class Fermentation(BatchBioreactor):
    line = 'Fermentation'

    # trying to incorporate my model here
    # properties are assumed without noting the stoichiometry, especially for yield calculations

    mu_max = 0.2 # maximum sp growth rate
    ks = 2 # saturation constant
    kd = 0.0008 #cell death ratio
    m = 0.003 # maintanence coefficient
    sf = 150 # fed batch substrate

    # yields/ proportion of consumption

    yps = 0.1
    yxs = 0.5
    ypx = 0.2
    yo2 = 0.3 

    # properties to calculate oxygen dependency of the fed batch fermentation

    p = 101325
    r = 8.314
    temp = 298.15

    ko2 = 0.1 # oxygen half velocity constant
    kla = 0.005 # mass transfer coefficient 
    h2 = 33 # gas-liquid partition coefficient of O2
    mw = 32 # molecular weight of O2

    O2g = ((p * 0.205 * mw) / (r * temp))

In [ ]:
    def __init__(self, ID='', ins=None, outs=(), thermo=None, *, 
                 tau,  N=None, V=None, T=305.15, P=101325., Nmin=2, Nmax=36,
                 efficiency=0.9, iskinetic=False):
      
        # changes I have to impart here, I consider Number of reactors equals 1
        # can I manipulate the batchbioreactor code?

        BatchBioreactor.__init__(self, ID, ins, outs, thermo,
                                 tau=tau, N=N, V=V, T=T, P=P, Nmin=Nmin, Nmax=Nmax)
        
        self._load_components()
        self.iskinetic = iskinetic
        chemicals = self.chemicals
        self.hydrolysis_reaction = Reaction('Sucrose + Water -> 2Glucose', 'Sucrose', 1.00, chemicals)

        # in my case i need a nitrogen source to equate the yohimbine production. Ammonium salts?
        # balanced equation with stoichiometry ....
        # if yohimbine HCl is formed, maybe I can use NH4Cl
        # oxygen limitations are under consideration so I have to include that too

        self.fermentation_reaction = Reaction('Glucose + O2 + NH3 -> Yohimbine + 2CO2',  'Glucose', efficiency, chemicals)
        # reaction is neither accurate nor balanced. need to find suitable N2 source

        self.cell_growth_reaction = cell_growth = Reaction('Glucose -> Yeast', 'Glucose', 0.70, chemicals, basis='wt')
        cell_growth.basis = 'mol'

        # commenting out the next codes for now

        #if all([i in self.chemicals for i in ('FFA', 'DAG', 'TAG', 'Glycerol')]):
            #self.lipid_reaction = self.oil_reaction = ParallelReaction([
                #Reaction('TAG + 3Water -> 3FFA + Glycerol', 'TAG', 0.23, chemicals),
                #Reaction('TAG + Water -> FFA + DAG', 'TAG', 0.02, chemicals)
            #])
        #else:
            #self.lipid_reaction = None
        #self.efficiency = efficiency

In [ ]:
    def _calc_efficiency(self, feed, tau): # pragma: no cover
        # Get initial concentrations
        y, e, s, w = feed.indices(['Yeast',
                                   '146-48-5', # yohimbine
                                   '492-61-5',
                                   '7732-18-5'])
        mass = feed.mass
        F_vol = feed.F_vol
        z0 = [1, 100, 0, 0.001, 50]  # initial conditions for X, S, P, O2l, V

In [ ]:
        # the fed match model should be incorporated in here
        
        def F(t):
          return 0.1

        def model(z, t):
          [X, S, P, O2l, V] = z

          # compute coefficients
          mu = ((mu_max * S) / (ks + S)) * (O2l / (ko2 + O2l))
          

          qs = mu / yxs

          rX = ((mu - kd) * X)
          rS = (qs + m) * X
          rO2 = (yo2 * rS) + (kla * ((O2g / h2) - O2l))

          # the differential equations corresponding to mass balances

          dVdt = F(t)

          dXdt = rX - ((X * F(t)) / V)

          dSdt = ((F(t) / V) * (sf - S)) - rS

          dPdt = ((-F(t) *P) / V) + (ypx * mu * X)

          dOdt = ((F(t) / V) * O2l) + rO2
          
          return (dXdt, dSdt, dPdt, dOdt, dVdt)

       
        t = np.linspace(0, 50, 1000)
        sol = odeint(model, z0, t)
        X, S, P, O2l, V = sol.transpose()

        fig, ax1 = plt.subplots()

        ax2 = ax1.twinx()
        ax1.plot(t, X, 'g-')
        ax1.plot(t, S,'-')
        ax1.plot(t, P,'r-')
        ax1.plot(t, O2l,'y-')
        ax1.legend(['Cell Conc.',
                    'Substrate Conc.',
                    'Product Conc.',
                    'Oxygen Conc'])
        ax2.plot(t, V, 'b')
        ax2.legend(['Vol'])

        plt.title('Fed batch model')
        ax1.set_xlabel('Time [hrs]')
        ax1.set_ylabel('Concentration [g/liter]', color='g')
        ax2.set_ylabel('Vol [m3]', color='b')

In [ ]:
@property
    def efficiency(self):
        return self.fermentation_reaction.X
    @efficiency.setter
    def efficiency(self, efficiency):
        self.fermentation_reaction.X = efficiency